In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
from datetime import datetime,timedelta
from sklearn import preprocessing

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 6, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False  

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Data Preproces

In [2]:
full_data = pd.read_csv('./input/full_data_mode_encode.csv')
full_data = full_data.drop(['Unnamed: 0'], axis=1)

full_data.shape
full_data.head()

(9934208, 68)

,id,msno,name_Album more,name_Artist more,name_Concert,name_Discover Chart,name_Discover Feature,name_Discover Genre,name_Discover New,name_Explore,...,registration_month,registration_day,genre_ids1,genre_ids2,genre_ids3,genre_ids4,genre_ids5,genre_ids6,genre_ids7,genre_ids8
0,-1,9176,0,0,0,0,0,0,0,1,...,1,2,359,0,0,0,0,0,0,0
1,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1259,0,0,0,0,0,0,0
2,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1259,0,0,0,0,0,0,0
3,-1,19273,0,0,0,0,0,0,0,0,...,5,25,1019,0,0,0,0,0,0,0
4,-1,9176,0,0,0,0,0,0,0,1,...,1,2,1011,0,0,0,0,0,0,0


In [3]:
train = full_data[full_data.id == -1]
test = full_data[full_data.id != -1]

ids = test.id
train = train.drop(['id'], axis=1)
test = test.drop(['id', 'target'], axis=1)

del full_data

## Train

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
x_train = train.drop(['target'], axis=1)
y_train = train.target
x_test = test

del train
del test

x_train.shape
x_test.shape
x_train.head()

(7377418, 66)

(2556790, 66)

,msno,name_Album more,name_Artist more,name_Concert,name_Discover Chart,name_Discover Feature,name_Discover Genre,name_Discover New,name_Explore,name_Local playlist more,...,registration_month,registration_day,genre_ids1,genre_ids2,genre_ids3,genre_ids4,genre_ids5,genre_ids6,genre_ids7,genre_ids8
0,9176,0,0,0,0,0,0,0,1,0,...,1,2,359,0,0,0,0,0,0,0
1,19273,0,0,0,0,0,0,0,0,1,...,5,25,1259,0,0,0,0,0,0,0
2,19273,0,0,0,0,0,0,0,0,1,...,5,25,1259,0,0,0,0,0,0,0
3,19273,0,0,0,0,0,0,0,0,1,...,5,25,1019,0,0,0,0,0,0,0
4,9176,0,0,0,0,0,0,0,1,0,...,1,2,1011,0,0,0,0,0,0,0


In [13]:
knn = KNeighborsClassifier(n_neighbors=15, n_jobs=-1)

knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=15, p=2,
           weights='uniform')

In [14]:
y_test = knn.predict_proba(x_test)
y_train_pred = knn.predict_proba(x_train)

In [15]:
sub = pd.DataFrame()
sub['id'] = ids
sub['target'] = y_test[:, 1]
sub.head()

,id,target
7377418,0,0.333333
7377419,1,0.600000
7377420,2,0.400000
7377421,3,0.466667
7377422,4,0.466667


In [18]:
sub.to_csv('./submit/knn_2.csv', index=False, float_format = '%.6f')

In [19]:
y_train = pd.DataFrame(y_train_pred[:, 1])
y_train.to_csv('./submit/knn_train.csv', index=False, float_format = '%.6f')

In [4]:
from sklearn.ensemble import AdaBoostClassifier

In [7]:
ada = AdaBoostClassifier(n_estimators =300, learning_rate=0.15)

ada.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.15, n_estimators=300, random_state=None)

In [11]:
y_test = ada.predict_proba(x_test)
y_train_pred = ada.predict_proba(x_train)

In [9]:
sub = pd.DataFrame()
sub['id'] = ids
sub['target'] = y_test[:, 1]
sub.head()

,id,target
7377418,0,0.503618
7377419,1,0.503425
7377420,2,0.494474
7377421,3,0.492669
7377422,4,0.492219


In [10]:
sub.to_csv('./submit/ada_2.csv', index=False, float_format = '%.6f')

In [12]:
y_train = pd.DataFrame(y_train_pred[:, 1])
y_train.to_csv('./submit/ada_train.csv', index=False, float_format = '%.6f')